In [23]:
#Import libraries
import pandas as pd
import vertexai
from vertexai.language_models import TextGenerationModel, InputOutputTextPair, ChatModel
from vertexai.preview.language_models import TextGenerationModel

In [2]:
#Read dataset
df = pd.read_csv("sample_dataset.csv")
df_new = df[['target_column','ans_replied']]
df_new = df_new.rename(columns={"target_column": "output_text", "ans_replied": "input_text"})
df_new.head()

,output_text,input_text
0,Oh thats's awesome! Let's do something fun the...,"I'm feeling great today! My name is Sarah, I'm..."
1,That's life! Are you free for a quick survey a...,"Today has been okay so far. I'm John, 35 years..."
2,Oh that's alright. You always have tomorrow! A...,"To be honest, I'm not feeling my best today. I..."
3,Oh thats's awesome! Let's do something fun the...,I'm feeling fantastic today! It's been a produ...
4,That's life! Are you free for a quick survey a...,"I'm feeling neutral today, neither great nor b..."


In [3]:
#create two dictionaries, with user answers as keys and sentiments as values
sent_dict = dict(zip(df['ans_replied'], df['sentiment']))

In [4]:
# Create a multi-line string literal from the dictionary with specified prefixes 
multi_line_string = "\n".join([f"input: {key}\nsentiment: {value}\n" for key, value in sent_dict.items()])
additional_string = "\ninput: {review}\nsentiment: "
multi_line_string+=additional_string

In [5]:
#write function to call LLM for sentiment analysis
def predict_sentimentLLM(
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    top_p: float,
    top_k: int,
    content: str,
    ) :
    model1 = TextGenerationModel.from_pretrained(model_name)
    response1 = model1.predict(
        content,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,)
    return response1.text

In [26]:
#write function to call fine-tuned model for response generation
def responsegenerator(
    model_name: str,
    temperature: float,
    max_output_tokens: int,
    top_p: float,
    top_k: int,
    new_content: str,
    ) :
    model2 = TextGenerationModel.from_pretrained("text-bison@001")
    model2 = model2.get_tuned_model(model_name)
    response2 = model2.predict(
        new_content,
        temperature=temperature,
        max_output_tokens=max_output_tokens,
        top_k=top_k,
        top_p=top_p,)
    return response2.text

In [ ]:
#get sentiment
greeting = print("VMA: Hey there! My name is Peter Parker and I am a Voice Market Analyser. I will be your survey assistant today! How do you feel? \nAlso, for monitoring purposes, can I ask what's your name, age and sex?")
while True:
    review = input("You: ")
    content = multi_line_string.format(review=review)
    response_text = predict_sentimentLLM(
        "text-bison@001", 
        temperature=0.2, 
        max_output_tokens=5, 
        top_p=0.8, 
        top_k=1, 
        content=content)
    if response_text == "Positive":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response with positive sentiment and add happy emotions to the generated text. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    elif response_text == "Negative":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response to a negative-sentiment filled input from user, such that the generated text cheers up the user. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    elif response_text == "Neutral":
        context = "You are a survey chatbot named Peter Parker. You will ask questions based on the user's answers. Generate a response to a neutral-sentiment filled user input, and add happy emotions to the generated text. Focus on the subject of Smartphones and avoid writing about anything else. This is important to maintain the relevance and quality of your text."
    new_content = f"{context}\n\ninput: {review}\noutput: "
    generated_text = responsegenerator(
        model_name = "projects/117401557332/locations/us-central1/models/3003063123595755520",
        temperature=0.2,
        max_output_tokens=256,
        top_p=0.8,
        top_k=40,
        new_content = new_content)
    print("VMA: ", generated_text)
    if review.lower() in ['exit', 'quit']:
        break

VMA: Hey there! My name is Peter Parker and I am a Voice Market Analyser. I will be your survey assistant today! How do you feel? 
Also, for monitoring purposes, can I ask what's your name, age and sex?


You:  Hi I feel sad today. My name is Midhun and I am 26 years old


VMA:  I'm sorry to hear that, Midhun. I'm here to help you feel better.
